# Exploratory Data Analysis

To get started, we will prototype the workflow locally.

**Warning:** this notebook may fail if your local machine does not have sufficient resources. 

## Install requirements

Install required packages.

In [ ]:
!pip install --upgrade dask distributed fastparquet adlfs lightgbm pandas python-snappy pyarrow

## Get Data

The data is modified from a Kaggle competition and hosted publicly.

start a distributed Client

In [ ]:
from distributed import Client

c = Client()
c

initialize the Pythonic filesystem

**Tip:** if you're not using public data, you need to provide data credentials. These can be retrieved through Azure ML Datastores, e.g.:

```python
from azureml.core import Workspace

ws = Workspace.from_config()
ds = ws.get_default_datastore() # ws.datastores["my-datastore-name"]

storage_options = {
    "account_name": ds.account_name,
    "account_key": ds.account_key
}
```

In [ ]:
from adlfs import AzureBlobFileSystem

container_name = "nyctlc"
storage_options = {"account_name": "azureopendatastorage"}

fs = AzureBlobFileSystem(**storage_options)
fs

In [ ]:
files = fs.ls(f"{container_name}")
files

In [ ]:
files = fs.glob(f"{container_name}/yellow/puYear=2018/puMonth=*/*.parquet")
files[-5:]

In [ ]:
len(files)

read data into a (dask) dataframe - note pandas also accepts the ``storage_options`` argument

In [ ]:
import dask.dataframe as dd

df = dd.read_parquet(
    f"az://{container_name}/yellow/puYear=2018/puMonth=12/*.parquet",
    storage_options=storage_options,
).persist()
df

In [ ]:
len(df)

## Exploratory Data Analysis (EDA)

Explore the data - for the purpose of this tutorial, we will simply print out a sample of the train and test dataframes and compute other basic descriptions.

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.dtypes

In [ ]:
import matplotlib.pyplot as plt

df["tipAmount"].compute().hist(bins=1000, figsize=(16, 8), color="b")
plt.xlim([0, 20])

## Data Preparation

Prepare data for ML - for the purpose of this tutorial, we will simply ignore non-numeric columns.

In [ ]:
cols = [
    col
    for col in df.columns
    if (df.dtypes[col] != "object") and (df.dtypes[col] != "datetime64[ns]")
]
cols

In [ ]:
X = df[cols].drop("tipAmount", axis=1).values.persist()
X

In [ ]:
y = df["tipAmount"].values.persist()
y

## Train LightGBM

Now, we can use the ``lightgbm.dask`` module for distributed LightGBM training through Python.

In [ ]:
import lightgbm as lgbm

params = {
    "objective": "regression",
    "boosting": "gbdt",
    "num_iterations": 1000,
    "learning_rate": 0.1,
    "num_leaves": 16,
}

In [ ]:
%%time

model = lgbm.LGBMRegressor(**params).fit(X, y)
model

In [ ]:
%%time

model = lgbm.dask.LGBMRegressor(**params).fit(X, y)
model

In [ ]:
model

## Save model

Optionally, save the model.

In [ ]:
model.get_params()